# Introduction

### Brief Overview

In this notebook, one can find answers to the following questions:
* What active learning is?
* How to use implementations of active learning strategies from `dsawl` package?
* How do $\varepsilon$-greedy active learning perform relatively random selection?

### References

* An article that contains review of approaches to active learning: [Yang, 2017](https://arxiv.org/pdf/1702.08540.pdf);
* An article about EG-Active algorithm: [Bouneffouf, 2014](https://arxiv.org/abs/1408.2196).

# General Preparations

### Import Statements

In [1]:
import math
from copy import copy

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

from dsawl.active_learning.pool_based_sampling import EpsilonGreedyPickerFromPool

### Notebook-level Settings

In [2]:
np.random.seed(361)

In [3]:
sns.set()

### User-defined Settings

In [4]:
# It is not a good practice to store binary files
# (like PNG images) in a Git repository, but for
# your local use you can set it to `True`.
draw_plots = False

# Active Learning Overview

In [5]:
# Extract necessary info from docstring
# in order to avoid copying and pasting.
import dsawl.active_learning.pool_based_sampling as pbs
print(pbs.__doc__.split('\n\n')[1])

Active learning setup assumes that, given a model and a training set,
it is possible to extend the training set with new labelled examples
and the goal is to do it with maximum possible improvement of model
quality subject to constraint on how many new examples can be added.
Further, pool-bases sampling means that new examples come from
a fixed and known set of initially unlabelled examples, i.e., the task
is to choose objects to be studied, not to synthesize them arbitrarily.


# Dataset Generation

Make a dataset that is involved in further examples.

In [6]:
dimensionality = 2
lower_bound = -2
upper_bound = 2
pool_size = 300

In [7]:
X_train_initial = np.array(
    [[1, -1],
     [2, -2],
     [3, -3],
     [-1, -1],
     [-2, -2],
     [-3, -3],
     [0, 1],
     [0, 2],
     [0, 3]]
)

In [8]:
X_new = np.random.uniform(
    lower_bound, upper_bound, size=(pool_size, dimensionality)
)

In [9]:
X_hold_out = np.random.uniform(
    lower_bound, upper_bound, size=(pool_size, dimensionality)
)

In [10]:
def compute_target(X: np.ndarray) -> np.ndarray:
    """
    Compute class label for a simple classification problem where
    2D plane is split into three regions by rays such that they
    start from the origin and an angle between any pair of them
    has 120 degrees.
    
    :param X:
        coordinates of points from the plane
    :return:
        labels of regions where points are located
    """
    
    def compute_target_for_row(x: np.ndarray) -> int:
        if x[0] > 0:
            return 1 if x[1] - math.tan(math.radians(30)) * x[0] > 0 else 2
        else:
            return 1 if x[1] + math.tan(math.radians(30)) * x[0] > 0 else 3
        
    y = np.apply_along_axis(compute_target_for_row, axis=1, arr=X)
    return y

In [11]:
y_train_initial = compute_target(X_train_initial)
y_new = compute_target(X_new)
y_hold_out = compute_target(X_hold_out)

In [12]:
if draw_plots:
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111)
    for label, color in zip(range(1, 4), ['b', 'r', 'g']):
        curr_X = X_train_initial[y_train_initial == label, :]
        ax.scatter(curr_X[:, 0], curr_X[:, 1], c=color, marker='D')
    for label, color in zip(range(1, 4), ['b', 'r', 'g']):
        curr_X = X_new[y_new == label, :]
        ax.scatter(curr_X[:, 0], curr_X[:, 1], c=color)

# Step-by-Step Tutorial

Instances of `EpsilonGreedyPickerFromPool` have two initialization arguments: `scorer` and `exploration_probability`. Let us discuss both of them.

Argument named `scorer` defines an internal entity that ranks new objects by usefullness of their labels. The higher the rank, the more valuable a label of an object is. As for technical implementation, all scorers are instances of classes that inherit from these one class:
`dsawl.active_learning.pool_based_sampling.BaseScorer`.

Any instance that satisfies above condition can be passed as `scorer`. However, the easiest and the safest way to pass value of `scorer` is to pass a string that can be recognized as a name of pre-defined scorer.

For classification, supported strings are:
* 'confidence' — the $i$-th object has score $\max_{j} \hat{p}_{ij}$ where $\hat{p}_{ij}$ is estimated (predicted) probability that the $i$-th object is an object of $j$-th class;
* 'margin'  — the $i$-th object has score $\max_{j} \hat{p}_{ij} - \max_{j \ne \hat{y}_i} \hat{p}_{ij}$ where $\hat{y}_i$ is predicted class of the $i$-th object, i.e., $\hat{y}_i = \arg \max {j} \hat{p}_{ij}$;
* 'entropy' — the $i$-th object has score $\sum_{j} \hat{p}_{ij} \log \hat{p}_{ij}$;
* 'divergence' — the $i$-th object has score $\sum_{k}D_{KL}(\hat{p}_{ijk} \, \Vert \, \overline{p}_{ij})$ where there is a committee (i.e., list) of classifiers indiced by $k$, $\hat{p}_{ijk}$ is predicted by the $k$-th classifier probability that the $i$-th object is an object of $j$-th class, $\overline{p}_{ij}$ is the average of all $\hat{p}_{ijk}$ over $k$, and $D_{KL}$ is Kullback-Leibler divergence between $\hat{p}_{ijk}$ and $\overline{p}_{ij}$ (both are considered to be distributions of class label $j$).

For regression, supported strings are:
* 'predictions_variance' — the $i$-th object has score $\mathrm{Var}_k \hat{y}_{ik}$ where there is a committee of regressors indiced by $k$ and $\hat{y}_{ik}$ is predicted by the $k$-th regressor target value for the $i$-th object;
* 'target_variance' — the $i$-th object has score that is equal to an estimate of target's variance on it: $\max(\hat{y^2}_i - \hat{y}_i^2, 0)$ where there is a pair of regressors and the first one predicts target itself, whereas the second one predicts squared target.

All of the above strings define scoring function, but do not define tools of a scorer. Here the word 'tools' means a classifier, a pair of regressors, or a committee of classifiers or regressors. Such tools must be passed explicitly. It can be done either with `set_tools` method (properly trained tools are required) or with `update_tools` method (just one estimator is needed, but training data must be provided too).

Below cells illustrate how to pass `scorer`.

In [13]:
picker = EpsilonGreedyPickerFromPool(scorer='margin')
clf = RandomForestClassifier()
clf.fit(X_train_initial, y_train_initial)
picker.set_tools(clf)

In [14]:
picker = EpsilonGreedyPickerFromPool(scorer='margin')
picker.update_tools(X_train_initial, y_train_initial, RandomForestClassifier())

Now, go to `exploration_probability` argument. It can be either float or a list of floats. If it is a float, it produces no problem in a short run, but if there are lots of calls, it is better to start with high exploration probability and then decrease it gradually. List of float as `exploration_probability` allows doing so, but also it imposes a limitation on total number of calls — it can not be higher than the length of the list.

In [15]:
picker = EpsilonGreedyPickerFromPool(scorer='margin', exploration_probability=[0.5, 0.4, 0.3, 0.2, 0.1])
picker.update_tools(X_train_initial, y_train_initial, RandomForestClassifier())

Usage of a created instance is as simple as the next cell.

In [16]:
indices = picker.pick_new_objects(X_new, n_to_pick=3)
X_new[indices, :]

array([[-0.06107047, -1.73046259],
       [-0.50815302,  0.92325141],
       [-0.2172508 , -1.24209527]])

# Illustrative End-to-End Example

Here $\varepsilon$-greedy strategy is compared with a benchmark based on random selection from a pool.

In [17]:
clf = RandomForestClassifier(n_estimators=20, random_state=361)

In [18]:
max_n_points_to_explore = 100

In [19]:
epsilon = 0.1
scorer = 'margin'

In [20]:
def report_accuracy_of_benchmark(
        n_new_points: int,
        clf: BaseEstimator,
        X_train_initial: np.ndarray, y_train_inital: np.ndarray,
        X_new: np.ndarray, y_new: np.ndarray,
        X_hold_out: np.ndarray, y_hold_out: np.ndarray
        ) -> float:
    """
    Compute accuracy of approach where `n_new_points` objects
    are picked from a pool at random, without active learning.
    """
    X_train = np.vstack((X_train_initial, X_new[:n_new_points, :]))
    y_train = np.hstack((y_train_initial, y_new[:n_new_points]))
    clf.fit(X_train, y_train)
    y_hold_out_hat = clf.predict(X_hold_out)
    return accuracy_score(y_hold_out, y_hold_out_hat)

In [21]:
def report_accuracy_of_epsilon_greedy_strategy(
        n_new_points: int,
        clf: BaseEstimator,
        epsilon: float,
        scorer: str,
        X_train_initial: np.ndarray, y_train_inital: np.ndarray,
        X_new: np.ndarray, y_new: np.ndarray,
        X_hold_out: np.ndarray, y_hold_out: np.ndarray
        ) -> float:
    """
    Compute accuracy of epsilon-greedy approach to active
    learning.
    """
    X_train = copy(X_train_initial)
    y_train = copy(y_train_inital)
    clf.fit(X_train, y_train)
    picker = EpsilonGreedyPickerFromPool(
        scorer, exploration_probability=epsilon
    )
    picker.set_tools(clf)
    for i in range(n_new_points):
        indices = picker.pick_new_objects(X_new, n_to_pick=1)
        X_train = np.vstack((X_train, X_new[indices, :]))
        y_train = np.hstack((y_train, y_new[indices]))
        picker.update_tools(X_train, y_train)
        X_new = np.delete(X_new, indices, axis=0)
        y_new = np.delete(y_new, indices)
    clf = picker.get_tools()
    y_hold_out_hat = clf.predict(X_hold_out)
    return accuracy_score(y_hold_out, y_hold_out_hat)

In [22]:
benchmark_scores = [
    report_accuracy_of_benchmark(
        n, clf,
        X_train_initial, y_train_initial, X_new, y_new,
        X_hold_out, y_hold_out
    )
    for n in range(1, max_n_points_to_explore + 1)
]
sum(benchmark_scores)

93.703333333333276

In [23]:
epsilon_greedy_scores = [
    report_accuracy_of_epsilon_greedy_strategy(
        n, clf, epsilon, scorer,
        X_train_initial, y_train_initial, X_new, y_new,
        X_hold_out, y_hold_out
    )
    for n in range(1, max_n_points_to_explore + 1)
]
sum(epsilon_greedy_scores)

96.056666666666743

In [24]:
if draw_plots:
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111)
    ax.plot(benchmark_scores)
    ax.plot(epsilon_greedy_scores, c='g')

To conclude, it can be seen that there is a gain from usage active learning instead of selecting objects randomly.